<a href="https://colab.research.google.com/github/SiRui-Lu/Addversarial_attacks_On_defence_tools/blob/main/PCA_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade -q adversarial-robustness-toolbox==1.3.3

     |████████████████████████████████| 655kB 2.8MB/s 
     |████████████████████████████████| 6.8MB 7.7MB/s 
     |████████████████████████████████| 21.0MB 63.9MB/s 
     |████████████████████████████████| 235kB 53.6MB/s 
     |████████████████████████████████| 747kB 46.7MB/s 


# Dataset preprocssing: Normalisation

Tring to scale the distribution to a mean of zero and a standard deviation of one for each feature value

In [62]:
from sklearn.datasets import load_breast_cancer
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
breast = load_breast_cancer()
#-----------------------------
breast_data = breast.data
breast_label = breast.target
#-----------------------------
labels = np.reshape(breast_label,(569,1))
ohe = OneHotEncoder() #in order to feed the label into NN, we have to change it to binary code ####
y = ohe.fit_transform(labels).toarray()

#concatenate data and label together
final_breast_data = np.concatenate([breast_data,labels],axis=1)

#converting np array to dataframe
breast_dataset = pd.DataFrame(final_breast_data)

#adding features label into the dataframe
features = breast.feature_names
features_labels = np.append(features,'label')
breast_dataset.columns = features_labels
# changing the label 0/1 to the real value
# breast_dataset['label'].replace(0, 'Benign',inplace=True)
# breast_dataset['label'].replace(1, 'Malignant',inplace=True)



array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
        1.189e-01],
       [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e-01, 2.750e-01,
        8.902e-02],
       [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e-01, 3.613e-01,
        8.758e-02],
       ...,
       [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
        7.820e-02],
       [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e-01, 4.087e-01,
        1.240e-01],
       [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
        7.039e-02]])

In [59]:
from sklearn.preprocessing import StandardScaler
x = breast_dataset.loc[:, features].values
x = StandardScaler().fit_transform(x) # normalizing the features
print(np.mean(x),np.std(x)) #each feature of your data should be normally distributed such that it will scale the distribution to a mean of zero and a standard deviation of one

#converting normalised feature data into the dataframe
feat_cols = ['feature'+str(i) for i in range(x.shape[1])]
normalised_breast = pd.DataFrame(x,columns=feat_cols)

X_max = np.max(x, axis=0)
X_min = np.min(x, axis=0)


-6.118909323768877e-16 1.0


# Model and classifier training



*   x is the preprocessed data
*   y is the binary value of labels





In [ ]:
import numpy as np
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score
# import matplotlib.pyplot as plt
import tensorflow as tf
import art
# Keras model generating
import keras
from keras.models import Sequential
from keras.layers import Dense
import numpy as np

from art.attacks.evasion import FastGradientMethod
from art.estimators.classification import KerasClassifier
from tensorflow.python.framework.ops import disable_eager_execution

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=40) # training and test dataset

disable_eager_execution()



# kersa model creating
model = Sequential()
model.add(Dense(20, input_dim=30, activation='relu'))
model.add(Dense(15, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.compile(
#     loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(lr=0.01), metrics=["accuracy"]
# )

#generating the KerasClassifier from ART based on previous NN model
classifier = KerasClassifier(model=model, clip_values=(X_min, X_max), use_logits=False)

classifier.fit(X_train,y_train,batch_size=64,nb_epochs=100)


Converting predicted data from binary value to labels

In [110]:
y_pred = classifier.predict(X_test)
#Converting predictions to label
def pre_to_label(y_pred):
  pred = list()
  for i in range(len(y_pred)):
      pred.append(np.argmax(y_pred[i]))
  return pred

#Converting one hot encoded test label to label
test = list()
for i in range(len(y_test)):
  test.append(np.argmax(y_test[i]))


label_y_pred = pre_to_label(y_pred)
print(label_y_pred)


[1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1]


Assess the accuracy of the trained classifier

In [117]:
from sklearn.metrics import accuracy_score
a = accuracy_score(label_y_pred,test)
print("Accuracy of unattacked model: {}%".format(a*100))

Accuracy of unattacked model: 98.24561403508771%


# Attacks

FGM

In [185]:
#generating attack samples
attack = FastGradientMethod(estimator=classifier, eps=0.3)
x_test_adv = attack.generate(x=X_test)

predictions = classifier.predict(x_test_adv)
p = pre_to_label(predictions)
attack_a = accuracy_score(p,test)
print("Accuracy on FGM test examples: {}%".format(attack_a * 100))

Accuracy on FGM test examples: 72.80701754385966%


Deep Fool

In [186]:
from art.attacks.evasion import DeepFool
adv_crafter = DeepFool(classifier)
# x_train_adv = adv_crafter.generate(X_train)
x_test_adv = adv_crafter.generate(X_test)

predictions = classifier.predict(x_test_adv)
p = pre_to_label(predictions)
attack_a = accuracy_score(p,test)
print("Accuracy on DeepFool attacked test examples: {}%".format(attack_a * 100))




DeepFool:   0%|          | 0/114 [00:00<?, ?it/s]

DeepFool:   1%|          | 1/114 [00:00<00:27,  4.06it/s]

DeepFool:  13%|█▎        | 15/114 [00:00<00:17,  5.60it/s]

DeepFool:  19%|█▉        | 22/114 [00:00<00:12,  7.47it/s]

DeepFool:  22%|██▏       | 25/114 [00:00<00:10,  8.73it/s]

DeepFool:  24%|██▎       | 27/114 [00:01<00:09,  9.03it/s]

DeepFool:  25%|██▌       | 29/114 [00:01<00:09,  9.35it/s]

DeepFool:  28%|██▊       | 32/114 [00:01<00:07, 10.59it/s]

DeepFool:  30%|██▉       | 34/114 [00:01<00:07, 11.23it/s]

DeepFool:  36%|███▌      | 41/114 [00:01<00:05, 14.04it/s]

DeepFool:  39%|███▉      | 45/114 [00:02<00:04, 15.50it/s]

DeepFool:  43%|████▎     | 49/114 [00:02<00:03, 16.47it/s]

DeepFool:  45%|████▍     | 51/114 [00:02<00:04, 13.93it/s]

DeepFool:  53%|█████▎    | 60/114 [00:02<00:03, 17.49it/s]

DeepFool:  64%|██████▍   | 73/114 [00:02<00:01, 22.18it/s]

DeepFool:  68%|██████▊   | 77/114 [00:03<00:01, 20.20it/s]

DeepFool:  76%|███████▋  | 87/114 [00:03<00:01, 

Accuracy on DeepFool attacked test examples: 18.421052631578945%


Basic Iterative Method (BIM)

In [140]:
from art.attacks.evasion import BasicIterativeMethod as bim
attack = bim(classifier, eps=0.3)
x_test_adv = attack.generate(x=X_test)

predictions = classifier.predict(x_test_adv)
p = pre_to_label(predictions)
attack_a = accuracy_score(p,test)
print("Accuracy on BIM test examples: {}%".format(attack_a * 100))

Accuracy on BIM test examples: 49.122807017543856%


C&W L2

In [142]:
from art.attacks.evasion import CarliniL2Method
attack = CarliniL2Method(classifier)
x_test_adv = attack.generate(x=X_test)

predictions = classifier.predict(x_test_adv)
p = pre_to_label(predictions)
attack_a = accuracy_score(p,test)
print("Accuracy on C&W L2 test examples: {}%".format(attack_a * 100))

C&W L_2: 100%|██████████| 114/114 [01:24<00:00,  1.35it/s]


Accuracy on C&W L2 test examples: 86.8421052631579%


Boundary attack

In [152]:
from art.attacks.evasion import BoundaryAttack
attack = BoundaryAttack(classifier)
x_test_adv = attack.generate(x=X_test,y=y_test)

predictions = classifier.predict(x_test_adv)
p = pre_to_label(predictions)
attack_a = accuracy_score(p,test)
print("Accuracy on BoundaryAttack test examples: {}%".format(attack_a * 100))


Boundary attack: 100%|██████████| 114/114 [09:56<00:00,  5.23s/it]

Accuracy on BoundaryAttack test examples: 96.49122807017544%


# PCA applied

PCA generator

In [178]:
from sklearn.decomposition import PCA

ncom = 4
pca_dataset = PCA(n_components=ncom)
pc_data = pca_dataset.fit_transform(x)

px_train, px_test, py_train, py_test = train_test_split(pc_data,y,test_size = 0.2)

X_max = np.max(pc_data, axis=0)
X_min = np.min(pc_data, axis=0)


array([16.31923323, 12.57319412, 10.11242619,  5.19425215])

PCA model

In [ ]:
pcamodel = Sequential()
pcamodel.add(Dense(8, input_dim=ncom, activation='relu'))
pcamodel.add(Dense(6, activation='relu'))
pcamodel.add(Dense(4, activation='relu'))
pcamodel.add(Dense(2, activation='softmax'))
pcamodel.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

pca_classifier = KerasClassifier(model=pcamodel, clip_values=(X_min, X_max), use_logits=False)

pca_classifier.fit(px_train,py_train,batch_size=64,nb_epochs=100)


Unattacked PCA model performance

In [180]:
pca_pre = pca_classifier.predict(px_test)
#Converting one hot encoded test label to label
pca_test = list()
for i in range(len(y_test)):
  pca_test.append(np.argmax(y_test[i]))

label_pca_pre = pre_to_label(pca_pre)

a1 = accuracy_score(label_y_pred,test)
print("Accuracy of unattacked pca model: {}%".format(a1*100))



Accuracy of unattacked pca model: 98.24561403508771%


Attacks on pca model

In [ ]:
#FGM
attack = FastGradientMethod(estimator=pca_classifier, eps=0.3)
x_test_adv = attack.generate(x=px_test)

predictions = pca_classifier.predict(x_test_adv)
p = pre_to_label(predictions)
attack_a2 = accuracy_score(p,pca_test)


#DeepFool
adv_crafter = DeepFool(pca_classifier)
# x_train_adv = adv_crafter.generate(X_train)
x_test_adv = adv_crafter.generate(px_test)

predictions = pca_classifier.predict(x_test_adv)
p = pre_to_label(predictions)
attack_a3 = accuracy_score(p,pca_test)


#BIM
attack = bim(pca_classifier, eps=0.3)
x_test_adv = attack.generate(x=px_test)

predictions = pca_classifier.predict(x_test_adv)
p = pre_to_label(predictions)
attack_a4 = accuracy_score(p,pca_test)


#CarliniL2
attack = CarliniL2Method(pca_classifier)
x_test_adv = attack.generate(x=px_test)

predictions = pca_classifier.predict(x_test_adv)
p = pre_to_label(predictions)
attack_a5 = accuracy_score(p,pca_test)


#BoundaryAttack
attack = BoundaryAttack(pca_classifier)
x_test_adv = attack.generate(x=px_test,y=py_test)

predictions = pca_classifier.predict(x_test_adv)
p = pre_to_label(predictions)
attack_a6 = accuracy_score(p,pca_test)




DeepFool:   0%|          | 0/114 [00:00<?, ?it/s]

DeepFool:  11%|█         | 12/114 [00:00<00:01, 52.79it/s]

DeepFool:  39%|███▊      | 44/114 [00:00<00:01, 62.65it/s]

DeepFool:  67%|██████▋   | 76/114 [00:00<00:00, 82.53it/s]

DeepFool:  79%|███████▉  | 90/114 [00:00<00:00, 73.89it/s]

DeepFool: 100%|██████████| 114/114 [00:01<00:00, 102.51it/s]


C&W L_2:   0%|          | 0/114 [00:00<?, ?it/s]

C&W L_2:   1%|          | 1/114 [00:00<01:34,  1.20it/s]

C&W L_2:   2%|▏         | 2/114 [00:01<01:27,  1.29it/s]

C&W L_2:   3%|▎         | 3/114 [00:02<01:21,  1.37it/s]

C&W L_2:   4%|▎         | 4/114 [00:02<01:17,  1.41it/s]

C&W L_2:   4%|▍         | 5/114 [00:03<01:20,  1.35it/s]

C&W L_2:   5%|▌         | 6/114 [00:04<01:17,  1.39it/s]

C&W L_2:   6%|▌         | 7/114 [00:05<01:21,  1.31it/s]

C&W L_2:   7%|▋         | 8/114 [00:05<01:15,  1.40it/s]

C&W L_2:   8%|▊         | 9/114 [00:06<01:17,  1.35it/s]

C&W L_2:   9%|▉         | 10/114 [00:07<01:14,  1.40it/s]

C&W L_2:  10%

In [ ]:
print("Accuracy on FGM test examples: {}%".format(attack_a2 * 100))
print("Accuracy on DeepFool attacked test examples: {}%".format(attack_a3 * 100))
print("Accuracy on BIM test examples: {}%".format(attack_a4 * 100))
print("Accuracy on C&W L2 test examples: {}%".format(attack_a5 * 100))
print("Accuracy on BoundaryAttack test examples: {}%".format(attack_a6 * 100))